# Golden Lotus Club
https://www.artofsword.com

In [12]:
# Installs
#!pip install firebase-admin
#!pip install web3
#!pip install python-dotenv


In [1]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore
import random
from dateutil.rrule import rrulestr
from dateutil.parser import parse
import datetime
import os
import firebase_admin
from firebase_admin import credentials, firestore
import os
from web3 import Web3
from dotenv import load_dotenv


os.environ["GCLOUD_PROJECT"] = "wallet-login-45c1c"

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

Authentication

In [2]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Create users with authentication

### Create new auth users

In [ ]:
# Create a list of users to add
users = "Andrei"
email = "ankisword@gmail.com"
password = '123456'

user_uuids = {}
# Loop through the users and create them with a default password

try:
  # Create the user with the email and password
  user_record = auth.create_user(
    email=email,
    password=password
  )
  user_uuids[user] = user_record.uid
  # Print the user ID
  print(f'Successfully created new user: {user_record.uid}')
except firebase_admin.auth.AuthError as e:
  # Handle any errors
  print(f'Error creating new user: {e}')

## Add users collection

In [ ]:
load_dotenv(os.path.join('..', '.env'))
INFURA_API_KEY = os.getenv('INFURA_API_KEY')

def create_new_account():
    # Create new web3 accounts for the users
    infura_url = f"https://polygon-mumbai.infura.io/v3/{os.getenv('INFURA_API_KEY')}"
    web3 = Web3(Web3.HTTPProvider(infura_url))

    if web3.isConnected():
        print("is connected")
    else:
        print("Wow. Something went wrong")

    return web3.eth.account.create()

## Add club collection

In [ ]:
# Create a collection for users
users_collection = db.collection("users")

user_clubs_roles = {}

account = create_new_account()
for user in users:
    user_data = {
            "name": user,
            "email": "ankisword@gmail.com",
            "age": 25,
            "occupation": "Coach",
            "is_active": True,
            "clubs_roles": [],
            "address": account.address,
            "privateKey": account.key.hex(),
        }

    user_ref = users_collection.document(user_uuids[user])
    user_ref.set(user_data)
    print(f"Successfully created user: {user_uuids[user]}")

    if user in ["Andrei"]:
            user_clubs_roles[user_uuids[user]] = [{"club_id": "2", "role": "owner"}]

# After creating all users, update the users with their clubs and roles
for user_uuid, clubs_roles in user_clubs_roles.items():
    user_ref = users_collection.document(user_uuid)
    user_ref.update({"clubs_roles": clubs_roles})


In [74]:
users = ["Andrei"]
user_uuids = {}
user_uid = "40WiH4RtOIgtJxGjwO6vadjAOem2"


for user in users:

    # Get the user record
    user_record = auth.get_user(user_uid)
    print(user)
    print("User Record:")
    print("UID:", user_record.uid)
    print("Email:", user_record.email)
    user_uuids[user] = user_record.uid

Andrei
User Record:
UID: 40WiH4RtOIgtJxGjwO6vadjAOem2
Email: ankisword@gmail.com


In [14]:
# Create a collection for users
users_collection = db.collection("users")

user_clubs_roles = {}

for user in users:
     #user_ref = users_collection.document(user_uuids[user])

     if user in ["Andrei"]:
            user_clubs_roles[user_uuids[user]] = [{"club_id": "2", "role": "owner"}]

user_clubs_roles

{'40WiH4RtOIgtJxGjwO6vadjAOem2': [{'club_id': '2', 'role': 'owner'}]}

In [78]:
users_collection = db.collection("users")


# Create a collection for clubs
clubs_collection = db.collection("clubs")

# Create a document for a club with some data fields
club_id = '2'
club_data = {
    "name": "Golden Lotus Club",
    "description": "A Fencing Club",
    "calendars": ["as.lexus@googlemail.com"],
}
club_ref = clubs_collection.document(club_id)
club_ref.set(club_data)


# Create a users subcollection under the club
users_subcollection = club_ref.collection("members")

# Add users to the users subcollection
for user_uuid, clubs_roles in user_clubs_roles.items():
    for role in clubs_roles:
        if role["club_id"] == "2":
            # Get user data from the users collection
            user_doc_ref = users_collection.document(user_uuid)
            user_snapshot = user_doc_ref.get()
            user_data = user_snapshot.to_dict()

            # Create the user data to be stored in the subcollection
            user_subcollection_data = {
                "user": user_uuid,
                "name": user_data["name"],
                "email": user_data["email"],
                "address": user_data["address"],
                "role": role["role"],
            }
            users_subcollection.document(user_uuid).set(user_subcollection_data)



### Create groups

In [15]:
user_clubs_roles

{'40WiH4RtOIgtJxGjwO6vadjAOem2': [{'club_id': '2', 'role': 'owner'}]}

In [16]:
clubs_collection = db.collection("clubs")

# The ID of the club under which the group will be created
club_id = "2"

club_ref = clubs_collection.document(club_id)

users_subcollection = club_ref.collection("members")



for user_uuid, clubs_roles in user_clubs_roles.items():
    print('users to be addres to ' + club_id)
    for role in clubs_roles:
        if role["club_id"] == "2":
            user_uuids = user_uuid
            
print(user_uuids)



users to be addres to 2
40WiH4RtOIgtJxGjwO6vadjAOem2


In [90]:


group_data = {
    "name": "Evening Group",  # the name of the group
    "member_uuids": user_uuids,  # the UUIDs of the group's members
    "event_ids": [  # list of events, where each event is an object with a calendarId and eventId
        {
            "calendarId": "as.lexus@gmail.com",  # replace with actual calendarId
            "eventId": "4d06bu8godice3gdt7qs8k3cg4"  # replace with actual eventId
        },
        # more events can go here...
    ],
}

# Create the group in the groups subcollection under the club
groups_subcollection = club_ref.collection("groups")
group_ref = groups_subcollection.document()  # generate a random ID for the group
group_ref.set(group_data)

print(f"Successfully created group: {group_ref.id} under club: {club_id}")




Successfully created group: hY9GGRuS8XWifG3AjFfx under club: 2


In [66]:

# The ID of the club under which the group will be created
club_id = "2"
club_ref = clubs_collection.document(club_id)

# Get all user UUIDs from the users collection
user_uuids = [doc.id for doc in users_collection.stream()]

# Select a random subset of users to be members of the group
member_uuids = random.sample(user_uuids, 10)  # choose 10 members randomly

group_data = {
    "name": "Evening Group",  # the name of the group
    "member_uuids": member_uuids,  # the UUIDs of the group's members
    "event_ids": [  # list of events, where each event is an object with a calendarId and eventId
        {
            "calendarId": "as.lexus@gmail.com",  # replace with actual calendarId
            "eventId": "7ud1orqm7edujuhrgdp5hfaaba"  # replace with actual eventId
        },
        # more events can go here...
    ],
}

# Create the group in the groups subcollection under the club
groups_subcollection = club_ref.collection("groups")
group_ref = groups_subcollection.document()  # generate a random ID for the group
group_ref.set(group_data)

print(f"Successfully created group: {group_ref.id} under club: {club_id}")


Successfully created group: pGl6jBT6HPZpKTUn6Yxl under club: 2


## Add events to members

In [4]:
googleCalendarId = "a5652177622e249042f44aa44801e1329bf0948e36326967cb03c1ea0b9198e0@group.calendar.google.com"


SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'

In [5]:
def import_events():
    # List to store events
    events_list = []

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    print('Getting the upcoming 20 events')
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                          maxResults=100, singleEvents=False).execute()
    events = events_result.get('items', [])

    for event in events:
        if 'recurrence' in event:  # if the event is a recurring event
            # Getting individual instances of the recurring event
            instances_result = service.events().instances(calendarId=googleCalendarId, eventId=event['id']).execute()
            instances = instances_result.get('items', [])
            events_list.extend(instances)  # Add the instances to the event list
        else:
            events_list.append(event)  # Add the non-recurring event to the event list

        
    events_list = sorted(events_list, key=lambda k: parse(k['start']['dateTime'] if 'dateTime' in k['start'] else k['start']['date'] + 'T00:00:00Z'))

    return events_list


events = import_events()


Getting the upcoming 20 events


In [ ]:

# Get the members subcollection under the club
members_subcollection = club_ref.collection("members")

# Specify calendarId and eventId
calendarId = "a5652177622e249042f44aa44801e1329bf0948e36326967cb03c1ea0b9198e0@group.calendar.google.com"

# Constants
MIN_SCORE = 0  # Define your minimum score here
MAX_SCORE = 100  # Define your maximum score here

for event in events[0:100]:
    eventId = event['id']  # get eventId from the current event
    # Get the start time of the event
    event_start = event['start'].get('dateTime', event['start'].get('date'))
    # Parse the start time to a datetime object
    event_date = parse(event_start)
    # Format the date to "yyyy-mm-dd"
    event_date_str = event_date.strftime('%Y-%m-%d')

    
    # Add attendance records to the attendance subcollection of each member
    for user_uuid in member_uuids:
        # Get the document reference for the member
        member_doc_ref = members_subcollection.document(user_uuid)
        
        # Create an attendance subcollection under the member
        attendance_subcollection = member_doc_ref.collection("attendance")

        # 90% chance they attended
        attended = random.choices([True, False], weights=[90, 10], k=1)[0]

        win = random.choices([True, False], weights=[50, 50], k=1)[0]

        if attended:
            # Prepare attendance data
            attendance_data = {
                "attended": attended,
                "calendarId": calendarId,
                "coefficient": None,
                "date": event_date_str,
                "eventId": eventId,
                "eventParentId": eventId.split('_')[0],  # splits eventId on '_' and gets the first part
                "score": random.randint(MIN_SCORE, MAX_SCORE) if attended else None,  # generate a random score if attended
                "win": win if attended else None,
                "paid": False,
            }

            # generate a document ID from "calendarId_eventId"
            document_id = f"{calendarId}_{eventId}"
            attendance_ref = attendance_subcollection.document(document_id)  
            attendance_ref.set(attendance_data)


### Add events to Firebase [events] collection

In [6]:



def get_next_occurrence(rrule_str, dtstart):
    rrule = rrulestr(rrule_str.replace('RRULE:', ''), dtstart=dtstart)
    next_occurrence = rrule.after(datetime.datetime.now(datetime.timezone.utc))
    return next_occurrence.isoformat()

def import_events():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                          maxResults=10, singleEvents=False).execute()
    events = events_result.get('items', [])

    if not firebase_admin._apps:
        firebase_cred = credentials.Certificate("../firebase-adminsdk.json")
        default_app = firebase_admin.initialize_app(firebase_cred)

    db = firestore.client()

    for event in events:
        is_recurring = 'recurrence' in event
        event_id = event['id']
        summary = event.get('summary', '')
        location = event.get('location', '')
        recurrence_rules = event.get('recurrence', []) if is_recurring else []
        start = event['start'].get('dateTime', event['start'].get('date'))

        next_occurrence = None
        if is_recurring and recurrence_rules:
            next_occurrence = get_next_occurrence(recurrence_rules[0], parse(start))
        else:
            next_occurrence = start

        event_info = {
            'id': event_id,
            'is_recurring': is_recurring,
            'recurrence_rules': recurrence_rules,
            'summary': summary,
            'location': location,
            'start': start,
            'next_occurrence': next_occurrence,
        }

        event_ref = db.collection('events').document(event_id)
        event_ref.set(event_info)
        print(event_info, ' added')


if __name__ == '__main__':
    import_events()


Getting the upcoming 10 events
{'id': '51b8npkppkudnimnbcs2dtjfml', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;WKST=MO;BYDAY=MO,WE,FR'], 'summary': 'Evening training', 'location': '', 'start': '2023-11-15T19:00:00+02:00', 'next_occurrence': '2023-11-15T19:00:00+02:00'}  added
{'id': '7ud1orqm7edujuhrgdp5hfaaba', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;WKST=MO;BYDAY=MO,WE'], 'summary': 'Evening traning', 'location': '', 'start': '2023-08-02T19:00:00+03:00', 'next_occurrence': '2023-08-23T19:00:00+03:00'}  added


In [ ]:
firestore_client = firestore.client()
# Create a collection named 'groups'
groups_ref = firestore_client.collection('groups')
# Create a document with ID 'group1' and some data
group1_ref = groups_ref.document('group1')
group1_ref.set({
    'name': 'Group 1',
    'members': ['Alice', 'Bob', 'Charlie']
})

# Create another document with ID 'group2' and some data
group2_ref = groups_ref.document('group2')
group2_ref.set({
    'name': 'Group 2',
    'members': ['David', 'Eve', 'Frank']
})